In [0]:
%pip install tweepy nltk

In [0]:
import tweepy

# Claves de la API de Twitter
consumer_key = "6mBwGXgNWojSH8qpVtOODWjwi"
consumer_secret = "DNu94CqxAAjCu4TUdhqoPzYW6wlHUR0C1FIMVT4VeefpeJOnIp"
access_token = "1875700400109518848-YQtauZALPoyi09ei08kuto7EDM3fLg"
access_token_secret = "nQVtQqxjY7ZYQ8Ql5xCpYcANXmvN95EFVnBgqUIQAK1YE"

# Autenticación
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# Verificar conexión
user = api.verify_credentials()
print(f"Conexión exitosa con la cuenta: {user.name}")


In [0]:
%python
import tweepy

# Set up your Twitter API v2 credentials
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAAe2xwEAAAAAQEI8DmQAmzpGxeEvx2myEg47Ofo%3Dq9Mpfd0oip4VA7bdMr2s2uPzFAPbJn6To14qS5GHjALg1bceQn'

# Create a client
client = tweepy.Client(bearer_token=bearer_token)

# Function to fetch tweets
def fetch_tweets(query, count=100):
    tweets = []
    response = client.search_recent_tweets(query=query, max_results=count, tweet_fields=['text'])
    for tweet in response.data:
        tweets.append(tweet.text)
    return tweets

# Example: Get 100 tweets about "machine learning"
tweets = fetch_tweets("machine learning", count=100)

# Display the first 5 tweets
for i, tweet in enumerate(tweets[:5], 1):
    print(f"{i}: {tweet}\n")

In [0]:
import re
from pyspark.sql import SparkSession

# Crear sesión de Spark
spark = SparkSession.builder.appName("TwitterMLlib").getOrCreate()

# Función para limpiar texto
def clean_tweet(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)  # Eliminar URLs
    tweet = re.sub(r"@\w+|#", "", tweet)  # Eliminar menciones y hashtags
    tweet = re.sub(r"[^\w\s]", "", tweet)  # Eliminar caracteres especiales
    tweet = re.sub(r"\d+", "", tweet)  # Eliminar números
    return tweet.lower().strip()

# Aplicar limpieza
cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]

# Convertir a DataFrame de Spark
df = spark.createDataFrame([(tweet,) for tweet in cleaned_tweets], ["text"])

# Mostrar los datos
df.show(5, truncate=False)


In [0]:
display(df)

In [0]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.clustering import KMeans

# Tokenización
tokenizer = Tokenizer(inputCol="text", outputCol="words")
tokenized_df = tokenizer.transform(df)

# Remover palabras vacías
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
filtered_df = remover.transform(tokenized_df)

# Vectorización con TF
hashing_tf = HashingTF(inputCol="filtered", outputCol="raw_features", numFeatures=20)
tf_df = hashing_tf.transform(filtered_df)

# Aplicar TF-IDF
idf = IDF(inputCol="raw_features", outputCol="features")
idf_model = idf.fit(tf_df)
tfidf_df = idf_model.transform(tf_df)

In [0]:
# Configurar KMeans
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=3, maxIter=10)
model = kmeans.fit(tfidf_df)

# Realizar predicciones
result = model.transform(tfidf_df)

# Mostrar resultados
result.select("text", "cluster").show(truncate=False)

In [0]:
display(result.select("text").filter(result.cluster == 0))

Databricks visualization. Run in Databricks to view.

In [0]:
display(result.select("text").filter(result.cluster == 1))

Databricks visualization. Run in Databricks to view.

In [0]:
display(result.select("text").filter(result.cluster == 2))

Databricks visualization. Run in Databricks to view.